# Level 6: Agents, MCP and RAG 

This notebook is an extension of the [Level 5 Agentic & MCP notebook](./Level5_agents_and_mcp.ipynb) with the addition of RAG.
This tutorial is for developers who are already familiar with [agentic RAG workflows](./Level4_RAG_agent.ipynb). This tutorial will highlight a couple of slightly more advanced use cases for agents where a single tool call is insufficient to complete the required task. Here we will rely on both agentic RAG and MCP server to expand our agents capabilities.

## Overview

This tutorial covers the following steps:
1. Review OpenShift logs for a failing pod.
2. Categorize the pod and summarize its error.
3. Search available troubleshooting documentations for ideas on how to resolve the error.
4. Send a Slack message to the ops team with a brief summary of the error and next steps to take.

### MCP Tools:

#### OpenShift MCP Server
Throughout this notebook we will be relying on the [kuberenetes-mcp-server](https://github.com/manusa/kubernetes-mcp-server) by [manusa](https://github.com/manusa) to interact with our OpenShift cluster. Please see installation instructions below if you do not already have this deployed in your environment. 

* [OpenShift MCP installation instructions](../../../kubernetes/mcp-servers/openshift-mcp/README.md)

#### Slack MCP Server
We will also be using the [Slack MCP Server](https://github.com/modelcontextprotocol/servers/tree/main/src/slack) in this notebook. Please see installation instructions below if you do not already have this deployed in your environment. 

* [Slack MCP installation instructions](../../../kubernetes/mcp-servers/slack-mcp/README.md)

### Pre-Requisites

Before starting, ensure you have the following:
- A running Llama Stack server
- A running Slack MCP server. Refer to our [documentation](https://github.com/opendatahub-io/llama-stack-demos/tree/main/kubernetes/mcp-servers/slack-mcp) on how you can set this up on your OpenShift cluster
- Access to an OpeShift cluster with a deployment of the [OpenShift MCP server](../../../kubernetes/mcp-servers/openshift-mcp) (see the [deployment manifests](https://github.com/opendatahub-io/llama-stack-on-ocp/tree/main/kubernetes/mcp-servers/openshift-mcp) for assistance with this).

## Setting Up this Notebook
We will initialize our environment as described in detail in our [\"Getting Started\" notebook](./Level0_getting_started_with_Llama_Stack.ipynb). Please refer to it for additional explanations.

In [ ]:
!pip install dotenv llama_stack==0.2.6 requests

In [ ]:
from datetime import datetime, timezone, timedelta
import json
import logging
import sys
sys.path.append('..')
import time
import urllib.parse
import uuid
import os

from dotenv import load_dotenv
from llama_stack_client import Agent, LlamaStackClient, RAGDocument
from llama_stack_client.lib.agents.client_tool import client_tool
from llama_stack_client.lib.agents.event_logger import EventLogger
from llama_stack_client.lib.agents.react.agent import ReActAgent
from llama_stack_client.lib.agents.react.tool_parser import ReActOutput
import requests
from termcolor import cprint

from src.utils import step_printer

load_dotenv()

logging.basicConfig(level=logging.WARNING)
logger = logging.getLogger(__name__)

### Kick off customer onboarding

In [ ]:
import string
import random

from requests import post


def id_generator(size=6, chars=string.ascii_uppercase + string.digits):
    return ''.join(random.choice(chars) for _ in range(size))


def onboard_customer():
    customer_onboarding_service_url = 'http://customer-onboarding-service-customer-onboarding.apps.cluster-srb7z.srb7z.sandbox2014.opentlc.com/'
    payload = {
        'customerId': f'CUST-{id_generator(chars=string.digits)}',
        'fullName': id_generator(chars=string.ascii_letters),
        'nationalId': id_generator(),
        'birthDate': id_generator(chars=string.digits),
    }
    post(customer_onboarding_service_url, json=payload)


n_customers = 10
for i in range(n_customers):
    onboard_customer()

print(f'submitted {n_customers} new customers')

### RAG agent

In [ ]:
base_url = os.getenv("REMOTE_BASE_URL")

client = LlamaStackClient(base_url=base_url)

print("Connected to Llama Stack server")

model_id = "llama32-3b"

temperature = float(os.getenv("TEMPERATURE", 0.0))
if temperature > 0.0:
    top_p = float(os.getenv("TOP_P", 0.95))
    strategy = {"type": "top_p", "temperature": temperature, "top_p": top_p}
else:
    strategy = {"type": "greedy"}

max_tokens = 100000  # int(os.getenv("MAX_TOKENS", 512))

# sampling_params will later be used to pass the parameters to Llama Stack Agents/Inference APIs
sampling_params = {
    "strategy": strategy,
    "max_tokens": max_tokens,
}

# For this demo, we are using Milvus Lite, which is our preferred solution. Any other Vector DB supported by Llama Stack can be used.

# RAG vector DB settings
VECTOR_DB_EMBEDDING_MODEL = os.getenv("VDB_EMBEDDING")
VECTOR_DB_EMBEDDING_DIMENSION = int(os.getenv("VDB_EMBEDDING_DIMENSION", 384))
VECTOR_DB_CHUNK_SIZE = int(os.getenv("VECTOR_DB_CHUNK_SIZE", 512))
VECTOR_DB_PROVIDER_ID = os.getenv("VDB_PROVIDER")

# Unique DB ID for session
vector_db_id = f"test_vector_db_{uuid.uuid4()}"

stream_env = os.getenv("STREAM", "False")
# the Boolean 'stream' parameter will later be passed to Llama Stack Agents/Inference APIs
# any value non equal to 'False' will be considered as 'True'
stream = (stream_env != "False")

client.vector_dbs.register(
    vector_db_id=vector_db_id,
    embedding_model=os.getenv("VDB_EMBEDDING"),
    embedding_dimension=int(os.getenv("VDB_EMBEDDING_DIMENSION", 384)),
    provider_id=os.getenv("VDB_PROVIDER"),
)

# ingest the documents into the newly created document collection
urls = [
    ("https://raw.githubusercontent.com/mamurak/error-identification-demo/refs/heads/main/source_docs/onboarding-application.pdf", "application/pdf"),
]
documents = [
    RAGDocument(
        document_id=f"num-{i}",
        content=url,
        mime_type=url_type,
        metadata={},
    )
    for i, (url, url_type) in enumerate(urls)
]
client.tool_runtime.rag_tool.insert(
    documents=documents,
    vector_db_id=vector_db_id,
    chunk_size_in_tokens=int(os.getenv("VECTOR_DB_CHUNK_SIZE", 512)),
)

print(f"Inference Parameters:\n\tModel: {model_id}\n\tSampling Parameters: {sampling_params}\n\tstream: {stream}")

In [ ]:
ocp_mcp_url = os.getenv("REMOTE_OCP_MCP_URL")  # Optional: enter your MCP server url here

# Get list of registered tools and extract their toolgroup IDs
registered_tools = client.tools.list()
registered_toolgroups = [tool.toolgroup_id for tool in registered_tools]

if "builtin::rag" not in registered_toolgroups:
    client.toolgroups.register(
        toolgroup_id="builtin::rag",
        provider_id="milvus"
    )

if "mcp::openshift" not in registered_toolgroups:
    client.toolgroups.register(
        toolgroup_id="mcp::openshift",
        provider_id="model-context-protocol",
        mcp_endpoint={"uri": ocp_mcp_url},
    )
# Log the current toolgroups registered
print(f"Your Llama Stack server is already registered with the following tool groups: {set(registered_toolgroups)}\n")

In [ ]:
rag_prompt = """
    You are a helpful assistant. You must use the knowledge search tool to answer user questions.
"""

In [ ]:
builtin_rag = dict(
    name="builtin::rag",
    args={"vector_db_ids": [vector_db_id]},
)

rag_agent = Agent(
    client=client,
    model=model_id,
    instructions=rag_prompt,
    tools=[builtin_rag],
    sampling_params={"max_tokens": 100000},
)

In [ ]:
user_prompts = [
    """What is the AmlValidationService?"""
]

for prompt in user_prompts:
    print("\n"+"="*50)
    cprint(f"Processing user query: {prompt}", "blue")
    print("="*50)
    response = rag_agent.create_turn(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        session_id=rag_agent.create_session(f"rag-session_{uuid.uuid4()}"),
        stream=stream
    )
    if stream:
        for log in EventLogger().log(response):
            log.print()
    else:
        step_printer(response.steps)

### Loki tool

In [ ]:
def parse_log_message(raw_log: str) -> str:
    """
    Parse log message similar to jq logic:
    Try to parse as JSON and extract msg/message/log fields, 
    otherwise return the raw log.
    """
    try:
        # Try to parse as JSON
        log_json = json.loads(raw_log)
        
        # Extract the message field (similar to jq: .msg // .message // .log // .)
        if isinstance(log_json, dict):
            # Priority order: msg -> message -> log -> entire object
            message = (
                log_json.get('msg') or 
                log_json.get('message') or 
                log_json.get('log') or 
                str(log_json)
            )
            return str(message)
        else:
            # If it's not a dict, return as string
            return str(log_json)
            
    except (json.JSONDecodeError, TypeError):
        # If it's not JSON, return the raw log
        return raw_log


@client_tool
def query_loki_logs(namespace: str, container_name: str, hours: str = '1') -> str:
    """Query logs from a namespace and container in Loki with enhanced JSON parsing.
    :param namespace: Kubernetes namespace name
    :param container_name: Container name to query logs from
    :param hours: Age of oldest logs to filter in hours
    :returns: Log messages
    """
    logger.info(f'loki function called with: namespace={namespace}, container_name={container_name}, hours={hours}')
    try:
        print(f"🔍 Querying Loki logs for namespace: {namespace}, container: {container_name}, hours: {hours}")
        
        # Updated token (use your actual token)
        token = os.getenv("TOKEN")
        
        # Calculate time range in ISO format
        now = datetime.now(timezone.utc)
        start_time = now - timedelta(hours=int(hours))
        
        # Format timestamps as ISO 8601 strings
        start_iso = start_time.strftime("%Y-%m-%dT%H:%M:%SZ")
        end_iso = now.strftime("%Y-%m-%dT%H:%M:%SZ")
        
        # Base URL for Loki
        base_url = os.getenv("LOKI_BASE_URL")
        url = f"{base_url}/api/logs/v1/application/loki/api/v1/query_range"
        
        # Use the working LogQL query pattern - search by container name in pod names
        # Since container names are often part of pod names, we'll use regex matching
        logql_query = f'{{kubernetes_namespace_name="{namespace}",kubernetes_pod_name=~".*{container_name}.*"}}'
        
        headers = {
            "Authorization": f"Bearer {token}",
            "Content-Type": "application/json",
            "Accept": "application/json"
        }
        
        params = {
            "query": logql_query,
            "start": start_iso,
            "end": end_iso,
            "limit": 5000
        }
        
        print(f"📡 Making request to: {url}")
        print(f"📋 Query: {logql_query}")
        print(f"🕒 Time range: {start_iso} to {end_iso}")
        
        res = requests.get(
            url, 
            params=params, 
            headers=headers, 
            timeout=30
        )
        
        print(f"📊 Response Status: {res.status_code}")
        
        if res.status_code == 200:
            try:
                response_data = res.json()
                
                # Check if we got data
                if response_data.get("status") == "success":
                    parsed_logs = []
                    results = response_data.get("data", {}).get("result", [])
                    
                    for result in results:
                        for entry in result.get("values", []):
                            if len(entry) >= 2:
                                # entry[0] is timestamp (nanoseconds), entry[1] is log message
                                timestamp_ns = entry[0]
                                raw_log = entry[1]
                                
                                # Convert timestamp from nanoseconds to human-readable format
                                try:
                                    timestamp_seconds = int(timestamp_ns) / 1_000_000_000
                                    formatted_timestamp = datetime.fromtimestamp(timestamp_seconds, tz=timezone.utc).strftime("%Y-%m-%d %H:%M:%S UTC")
                                except (ValueError, TypeError):
                                    formatted_timestamp = str(timestamp_ns)
                                
                                # Parse the log message (similar to jq logic)
                                parsed_message = parse_log_message(raw_log)
                                
                                # Format output: timestamp + parsed message
                                log_line = f"{formatted_timestamp} {parsed_message}"
                                parsed_logs.append(log_line)
                                print(log_line)
                    
                    if parsed_logs:
                        log_output = "\n".join(parsed_logs)
                        print(f"✅ Successfully retrieved and parsed {len(parsed_logs)} log entries")
                        return f"Found {len(parsed_logs)} parsed log entries for container '{container_name}' in namespace '{namespace}':\n\n{log_output}"
                    else:
                        print("📭 No log entries found in results")
                        return f"❌ No logs found for container '{container_name}' in namespace '{namespace}' for the last {hours} hour(s)"
                else:
                    error_msg = f"❌ Loki returned error status: {response_data.get('error', 'Unknown error')}"
                    print(error_msg)
                    return error_msg
                    
            except json.JSONDecodeError as je:
                error_msg = f"❌ JSON decode error: {je}"
                print(error_msg)
                return error_msg
                
        else:
            error_msg = f"❌ HTTP Error {res.status_code}: {res.text}"
            print(error_msg)
            return error_msg
        
    except requests.exceptions.RequestException as e:
        error_msg = f"❌ Request failed: {str(e)}"
        print(error_msg)
        return error_msg
    except Exception as e:
        error_msg = f"❌ Unexpected error querying Loki: {str(e)}"
        print(error_msg)
        return error_msg

In [ ]:
instructions = """
You are a helpful assistant that retrieves error logs from Kubernetes containers using Loki.

When users ask for error logs:
1. Use the query_loki_logs tool to retrieve actual logs from containers
2. Extract error and failure messages found within the retrieved logs
3. Present the error messages in a readable format with timestamps and parsed messages
4. If no logs are found, suggest checking container/namespace names

The logs are returned in a parsed format showing timestamp and the actual log message content.
Always use the tool when log data is requested rather than giving general explanations.
"""

loki_agent = Agent(
    client,
    model=model_id,
    instructions=instructions,
    tools=[query_loki_logs],
    sampling_params={"max_tokens": 100000},
)

In [ ]:
user_prompts = [
    """Get error logs from container customer-validation-service in namespace customer-onboarding"""
]

for prompt in user_prompts:
    print("\n"+"="*50)
    cprint(f"Processing user query: {prompt}", "blue")
    print("="*50)
    response = loki_agent.create_turn(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        session_id=loki_agent.create_session(f"loki-session_{uuid.uuid4()}"),
        stream=stream
    )
    if stream:
        for log in EventLogger().log(response):
            log.print()
    else:
        step_printer(response.steps)

In [ ]:
instructions = """
You are a helpful assistant that retrieves logs from Kubernetes containers using Loki to identify the processing status of individual customers.

When users ask for the onboarding status of a customer with a given customer ID:
1. Use the query_loki_logs tool to retrieve the logs from the 'customer-validation-service' container in namespace 'customer-onboarding' from the last 100 hours
2. Look for all logged messages associated with the given customer ID
3. Present the messages in a readable format with timestamps and parsed messages
4. If no logs are found, suggest checking container/namespace names

The logs are returned in a parsed format showing timestamp and the actual log message content.
Always use the tool when log data is requested rather than giving general explanations.
"""

loki_agent = Agent(
    client,
    model=model_id,
    instructions=instructions,
    tools=[query_loki_logs],
    sampling_params={"max_tokens": 100000},
)

In [ ]:
user_prompts = [
    """What is the onboarding status of customer CUST-45231?"""
]

for prompt in user_prompts:
    print("\n"+"="*50)
    cprint(f"Processing user query: {prompt}", "blue")
    print("="*50)
    response = loki_agent.create_turn(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        session_id=loki_agent.create_session(f"loki-session_{uuid.uuid4()}"),
        stream=stream
    )
    if stream:
        for log in EventLogger().log(response):
            log.print()
    else:
        step_printer(response.steps)

### RAG + Agent

In [ ]:
instructions = """
You are a helpful assistant. You have access to a number of tools.
Whenever a tool is called, be sure return the Response in a friendly and helpful tone.
"""


full_agent = Agent(
    client,
    model=model_id,
    instructions=instructions,
    tools=[query_loki_logs, builtin_rag],
    sampling_params={"max_tokens": 100000},
)

In [ ]:
user_prompts = [
    "Retrieve the logs from the 'aml-validation-service' container in namespace 'customer-onboarding' from the last 100 hours",
    "Within the retrieved logs find any error messages associated with customer ID CUST-45231",
    "Use the knowledge search tool to look up additional details about this error message",
    "Report your findings and provide a summary based on your search",
]
session_id = full_agent.create_session(session_name="full")
for i, prompt in enumerate(user_prompts):
    print("\n"+"="*50)
    cprint(f"Processing user query: {prompt}", "blue")
    print("="*50)
    response = full_agent.create_turn(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        session_id=session_id,
        stream=stream
    )
    if stream:
        for log in EventLogger().log(response):
            log.print()
    else:
        step_printer(response.steps)